# オプティマイズ
この版はパラメトリック最適化にするつもり。
- 中心周波数
- バンド幅
- 分散補償
計 3パラメーター

必要なものは
- スクイージングレベルの測定
    - ポンプ光のON/OFF arduinoモジュール OK
    - スペアナのSCPIコントロール
    - もしかしたらロック角度の変調
- waveshaperコントロール OK
- パラメーター生成 全探索 OK

In [10]:
import numpy as np
from convinience import Spectrum, Band, synthesisSpectrum
from convinience import gaussian_profile, compensation, calc_totalPower ,attenuation
from mso58trans import MSO58Wrapper
from shutter import servo_shutter
from WSMethods import uploadProfile, getWSrange
from sklearn.decomposition import PCA
from csv import DictWriter
import datetime
import copy
from time import sleep

In [ ]:
# 各種設定周り ここにすべてまとめる

class Setting:
    def __init__(self):
        self.ws_ip                          = '169.254.6.8'
        self.ws_Freq_resolution             = 0.001 # 0.001THz = 1GHz
        self.ws_Freqrange                   = None
        self.optSpectrum : Spectrum         = None
        self.mso58_address                  = 'USB::0x0699::0x0530::C043144::INSTR'
        self.homodyne_port                  = 1
        self.fastFramePulseNum              = 5
        self.quadrature_method              = lambda volts: PCA().fit_transform(volts.reshape((-1,volts.shape[-1])))[:,0].reshape(volts.shape[:-1])
        # self.quadrature_method              = lambda volts: volts[:,100]
        self.targetLOPower                  = 1
        self.shutter_port                   = 'COM3'
        self.pumpShutterPort : int          = 2
        self.param_csv                      = f"{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}record.csv"


    def condition_approval(self):
        assert self.ws_Freqrange             is not None
        assert self.optSpectrum              is not None
        assert self.optSpectrum.powerdensity is not None
        assert self.optSpectrum.frequency    is not None
        assert self.optSpectrum.wavelength   is not None
        assert self.optSpectrum.phase        is not None
        assert self.fastFramePulseNum        is not None
        assert self.quadrature_method        is not None
        assert self.targetLOPower            is not None
        assert self.shutter_port             is not None
        assert self.pumpShutterPort          is not None
        assert self.param_csv                is not None


setting = Setting()

startFreq, endFreq    = getWSrange(setting.ws_ip)
setting.ws_Freqrange  = np.arange(startFreq, endFreq, setting.ws_Freq_resolution)

setting.optSpectrum                = Spectrum()
setting.optSpectrum.band           = Band()
setting.optSpectrum.frequency      = np.load('optFreq.npy')
setting.optSpectrum.powerdensity   = np.load('optSpectrum.npy')
setting.optSpectrum.phase          = np.zeros_like(setting.optSpectrum.frequency)

setting.condition_approval()

### 最適化フェーズ

In [43]:
class State:
    def __init__(self):
        self.current_param = None

state = State()

In [48]:

def quad(w, pulseNum, quadrature_func):
    frameLength = w.shape[1]
    frames      = w.shape[0]
    period = [int(i * frameLength/pulseNum) for i in range(pulseNum)]
    duration = int(frameLength/pulseNum)
    quadratures = np.zeros((frames, pulseNum))

    volts = np.zeros((frames,pulseNum,duration))

    for i in range(pulseNum):
        s = period[i]
        volts[:,i,:] = w[:,s:(s+duration)]

    quadratures = quadrature_func(volts)
    return quadratures


def evalation(quadratures):
    return quadratures.var()



def param_gen(state: State):
    for center_wavelength in np.linspace(1543,1547,40):
        for band_width in np.linspace(0.2,1,20):
            for dispersion in [0]:
                yield center_wavelength, band_width, dispersion



def loop():
    # optimizing loop
    for center_wavelength, band_width, dispersion in param_gen(state):
        ## new_LO_spectrum
        band = Band()
        band.frequency          = setting.ws_Freqrange
        filterSpectrum          = gaussian_profile(
                                    centerWavelength    =center_wavelength,
                                    bandWidth           =band_width, band=band,
                                    dispersion          =dispersion
                                )
        compensated_Spectrum    = compensation(
                                    setting.optSpectrum,
                                    targetSpectrum      =filterSpectrum,
                                    band                =band
                                )
        total_power = calc_totalPower(
                                    setting.optSpectrum,
                                    compensated_Spectrum
                                )
        ## (power adjustment)
        #filterSpectrum = attenuation(filterSpectrum, total_power - setting.targetLOPower)
        filterPort = np.ones(filterSpectrum.powerdensity.shape)
        r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

        ## pump off
        servo_shutter(port=setting.shutter_port).close([setting.pumpShutterPort])
        ## osc_com
        osc_handler = MSO58Wrapper(setting.mso58_address)
        osc_handler.push_single()
        t, w = None, None
        while True:
            try:
                t, w = osc_handler.transfer2byte(setting.homodyne_port)
                break
            except:
                sleep(1)
                continue
        ## quad
        quadratures = quad(w, setting.fastFramePulseNum, setting.quadrature_method)
        ## eval_shotnoise
        shot_var = quadratures.var()

        ## pump on
        servo_shutter(port=setting.shutter_port).open([setting.pumpShutterPort])
        ## osc_com
        osc_handler = MSO58Wrapper(setting.mso58_address)
        osc_handler.push_single()
        t, w = None, None
        while True:
            try:
                t, w = osc_handler.transfer2byte(setting.homodyne_port)
                break
            except:
                sleep(1)
                continue
        ## quad
        quadratures = quad(w, setting.fastFramePulseNum, setting.quadrature_method)
        ## eval_squeeze
        squeeze_var = quadratures.var()
        ## recording
        with open(setting.param_csv, 'a', newline = "") as f:
            headersCSV = ['center_wavelength', 'bandwidth', 'dispersion','LOpower', 'shotnoise', 'squeeze', 'diff']
            dict = {
                'center_wavelength': center_wavelength,
                'bandwidth'        : band_width,
                'dispersion'       : dispersion,
                'LOpower'          : total_power,
                'shotnoise'        : 10*np.log(shot_var),
                'squeeze'          : 10*np.log(squeeze_var),
                'diff'             : 10*np.log(squeeze_var) - 10*np.log(shot_var)
            }
            dictwriter_object = DictWriter(f, fieldnames=headersCSV)
            dictwriter_object.writerow(dict)
        print(dict)

In [49]:
loop()

TEKTRONIX,MSO64B,C043144,CF:91.1CT FV:1.34.8.1078



In [1]:
import numpy as np

In [8]:
np.random.randn(1000,30) * 10* np.sin( np.pi*np.linspace(0,1,30))

array([[-0.00000000e+00, -4.11558025e-01, -3.70537109e-01, ...,
         7.01548312e-01,  4.31496625e-01, -1.66272119e-15],
       [-0.00000000e+00,  1.27678304e+00, -1.40614160e+00, ...,
        -2.09467106e+00,  3.12234318e-01, -1.30105612e-15],
       [-0.00000000e+00, -4.01173228e-01, -1.55685328e+00, ...,
         2.25902390e+00, -1.76398872e-01,  5.73058395e-16],
       ...,
       [ 0.00000000e+00,  4.09368909e-01, -1.87631159e-01, ...,
        -5.97718365e-01, -1.78037253e+00, -1.08516704e-15],
       [ 0.00000000e+00, -1.12601798e+00,  4.78696900e-01, ...,
        -3.84147211e-01,  2.48201918e-01, -7.59658802e-16],
       [ 0.00000000e+00,  7.76224194e-02,  2.04023614e+00, ...,
        -3.09129852e+00, -8.96790622e-01,  8.98712994e-16]])

In [11]:
some = lambda volts: PCA().fit_transform(volts.reshape((-1,volts.shape[-1])))[:,0].reshape(volts.shape[:-1])

In [27]:
np.var(some(np.random.randn(1000,30) * 10* np.sin( np.pi*np.linspace(0,1,30))))

110.20457365517642